# Step 1: Set up the environment

In [109]:
import os
import google.generativeai as genai

if os.getenv("COLAB_RELEASE_TAG"):
   COLAB = True
   print("Running on COLAB environment.")
else:
   COLAB = False
   print("WARNING: Running on LOCAL environment.")


Running on COLAB environment.


In [110]:
# import colab specific lib to read user data (aka colab managed secrets)
from google.colab import userdata

In [111]:
# Initialize Google GenAI Client API with GOOGLE_API_KEY to be able to call the model.
# Note: GEMINI_API_KEY must be set as COLAB userdata before!
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [112]:
# Install additional libraries
%%capture
!pip install langchain-community
!pip install -qU langchain-text-splitters
!pip install chromadb

In [113]:
# Import additional libraries
from langchain_text_splitters import RecursiveCharacterTextSplitter
from chromadb import EphemeralClient
import requests
import re
import uuid

In [114]:
# Set default values for model, model parameters and prompt
DEFAULT_MODEL = "gemini-1.5-flash"
DEFAULT_CONFIG_TEMPERATURE = 0.9
DEFAULT_CONFIG_TOP_K = 1
DEFAULT_CONFIG_MAX_OUTPUT_TOKENS = 200
DEFAULT_SYSTEM_PROMPT = "Your are a friendly assistant"
DEFAULT_USER_PROMPT = ""

## Define helper functions

In [ ]:
# This will be the chromadb collection we use as a knowledge base. We do not need the in-memory EphemeralClient.
chromadb_collection = EphemeralClient().get_or_create_collection(name="default")

# The books are downloaded from the repository
def download_file(file_name: str) -> str:
  book_repository_github_user_name= 'TimWue'
  book_repository_github_repository_name= 'project-gutenberg-books'
  url = f"https://raw.githubusercontent.com/{book_repository_github_user_name}/{book_repository_github_repository_name}/main/{file_name}"
  r = requests.get(url)
  return r.text

# Prepare book content
def remove_useless_information(text: str) -> str:
  start_pattern = r"\*\*\* START OF (THE|THIS) PROJECT GUTENBERG EBOOK .{0,50} \*\*\*"
  end_pattern = r"\*\*\* END OF (THE|THIS) PROJECT GUTENBERG EBOOK .{0,50} \*\*\*"
  start_match = re.search(start_pattern, text)
  end_match = re.search(end_pattern, text)
  start_pos = start_match.end()
  end_pos = end_match.start()
  return text[start_pos:end_pos].strip()

# Prepare book content
def clean_whitespace(text: str) -> str:
    cleaned_text = text.replace('\\r\\n', ' ')
    return cleaned_text.strip()

# Prepare book content
def prepare_file_content(text: str) -> str:
  return clean_whitespace(remove_useless_information(text))

# Have a look into the knowledgebase
def peek_knowledgebase():
  print(chromadb_collection.peek())

In [116]:
def call_genai_model_for_completion(
        model_name: str = DEFAULT_MODEL,
        config_temperature:float = DEFAULT_CONFIG_TEMPERATURE,
        config_top_k: int = DEFAULT_CONFIG_TOP_K,
        config_max_output_tokens: int = DEFAULT_CONFIG_MAX_OUTPUT_TOKENS,
        system_prompt : str = DEFAULT_SYSTEM_PROMPT,
        user_prompt : str = DEFAULT_USER_PROMPT,
        verbose: bool = False
        ):

    if verbose:
        # print out summary of input values / parameters
        print(f'Generating answer for following config:')
        print(f'  - SYSTEM PROMPT used:\n {system_prompt}')
        print(f'  - USER PROMPT used:\n {user_prompt}')
        print(f'  - MODEL used:\n {model_name} (temperature = {config_temperature}, top_k = {config_top_k}, max_output_tokens = {config_max_output_tokens})')

    # create generation config
    model_config = genai.GenerationConfig(
        max_output_tokens=config_max_output_tokens,
        temperature=config_temperature,
        top_k=config_top_k
    )

    # create genai model with generation config
    genai_model = genai.GenerativeModel(
        model_name= model_name,
        generation_config= model_config
    )

    response = genai_model.generate_content([system_prompt, user_prompt])
    return response

In [117]:
def print_completion_result(completion_result, full:bool = False):

    # print out answer of genai model (aka text of response)
    print(f'\nANSWER of genAI model: \n')
    if full:
        print(completion_result)
    else:
        print(completion_result.text)

# Step 2: Configure the genAI models

In [118]:
GENERATION_MODEL = "gemini-1.5-flash"
EMBEDDING_MODEL = "models/text-embedding-004"

# Step 3: Configure retriever

In [133]:
DEFAULT_K = 3
DEFAULT_CHUNK_SIZE = 2000
DEFAULT_CHUNK_OVERLAP = 100

# Step 4: Define RAG Building Blocks

In [ ]:
# Get content of books. The content will already be cleansed.
def load_file_content(file_name: str) -> str:
  raw_content = download_file(file_name)
  return prepare_file_content(raw_content)


In [120]:
# Building Block "Chunking": Split the content into smaller chunks
def do_chunk(text: str) -> list[str]:
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=DEFAULT_CHUNK_SIZE,
      chunk_overlap=DEFAULT_CHUNK_OVERLAP,
      length_function=len,
  )
  return text_splitter.split_text(text=text)

In [121]:
# Building Block "Embedding": Create multi dimensional embeddings for a given chunk.
def do_embed(chunk: str) -> list[float]:
  return genai.embed_content(model=EMBEDDING_MODEL, content=chunk).get("embedding")

In [122]:
# Building Block "Knowledgebase": Store embeddings and the corresponding content in a vectorstore
def persist_embeddings(chunks: list[str], embeddings: list[float])-> None:
  ids = [str(uuid.uuid4()) for _ in chunks]
  chromadb_collection.add(ids=ids, documents=chunks, embeddings=embeddings)

In [123]:
# Building Block "Augmentation": Create an updated prompt by merging the original user input with the provided context
def augment(user_input: str, context: list[str]) -> str:
  prepared_context = "\n".join(context)
  augmented_prompt = f"""
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n{prepared_context}?\n
    Question: \n{user_input}\n

    Answer:
  """
  return augmented_prompt

In [ ]:
# Building Block "Top-k Fetching": Get the k semantically closest chunks to the user input from the knowledgebase
def do_top_k_fetching(user_input_embedding: list[float], k: int) -> list[str]:
  # Since we will do the fetching always only for one user_input,
  # instead of querying for multiple embeddings simultanously as allowed by the choma API,
  # we add the embeddings below to a list and return only the first document (chunk)
  return chromadb_collection.query(
      query_embeddings=[user_input_embedding],
      n_results=k,
  )["documents"][0]

In [125]:
# Building Block "Generation": Use the generation model to create a response
def generate_response(prompt: str) -> None:
  completion_result = call_genai_model_for_completion(
      model_name=GENERATION_MODEL,
      user_prompt=prompt,
  )
  print_completion_result(completion_result)

# Step 5: Create the ingestion pipeline

In [ ]:
def do_ingestion(file_names: list[str]) -> None:
  # Ingest file by file
  for file_name in file_names:
    # Load prepared book content
    file_content = load_file_content(file_name)

    # Chunk the content into smaller chunks
    chunks = do_chunk(file_content)

    # Embed the chunks
    embeddings = []
    for chunk in chunks:
      embedding = do_embed(chunk)
      embeddings.append(embedding)

    # Persist the embeddings and the chunks in the knowledgebase
    persist_embeddings(chunks, embeddings)


# Step 6: Perform ingestion

In [127]:
# Define file names to be ingested
file_names = ['study_in_scarlett.txt']

# Perform ingestion. Depending on the chunk_size this might take some minutes.
do_ingestion(file_names)

In [132]:
# Use helper function to peek into knowledgebase
peek_knowledgebase()

{'ids': ['9212d43a-d7be-4a8d-91cf-3f4ef354028e', 'd3611b6b-cf49-47e0-bdbb-01faa6b45f37', 'ebdc6b85-d142-4cd7-8bcb-13302a72cebe', 'f7719886-23cf-4421-a0c9-3e8b5406e716', 'b397fa20-7f24-4b86-b702-58bc231fe896', 'd05519ad-aede-4d9b-80a7-6dc94c549da1', '5a41a896-ead5-454b-8fa5-bb44d2603a2b', '957610fe-1e00-4166-9b85-921726415c16', '3b8fa243-c652-4e7c-a6d3-3b296abc57f7', 'e92bc402-8c7a-4f6f-bace-99e93cb246a4'], 'embeddings': array([[ 0.04196839,  0.03947523, -0.0369409 , ..., -0.01241391,
         0.05412204, -0.05153552],
       [ 0.04687842,  0.01152912, -0.0338999 , ..., -0.02374919,
         0.04071308, -0.05223513],
       [ 0.02813364,  0.03810108, -0.04354395, ..., -0.02314168,
         0.04599097, -0.07435063],
       ...,
       [ 0.02499893,  0.01168233, -0.04345908, ..., -0.04256863,
         0.02365425, -0.03552011],
       [ 0.01524693,  0.00685599, -0.04617725, ..., -0.04708918,
         0.04234932, -0.04377203],
       [ 0.01722268,  0.00676511, -0.03388088, ..., -0.02772887,

# Step 7: Create RAG pipeline

In [ ]:
def do_rag(user_input: str, verbose: bool = False) -> None:
  # TODO: Embed the user input
  user_input_embedding = None

  # TODO: "R" like "Retrieval": Get the k semantically closest chunks to the user input from the knowledgebase
  context = None
  if verbose:
    print(f'Retrieved context:\n {context}')

  # TODO: "A" like "Augmented": Create the augmented prompt
  augmented_prompt = None
  if verbose:
    print(f'Augmented prompt:\n {augmented_prompt}')

  # TODO: "G" like "Generation": Generate a response
  generate_response(prompt=augmented_prompt)


# Step 8: Perform RAG

In [ ]:
# Define user input. This should be a question regarding the ingested book
user_input = "Lucy noticed a number on the ceiling when taking breakfast. which number was written into the ceiling?" # The answer should contain the number "28"

# TODO: Perform retrieval
